In [ ]:
import panel as pn
pn.extension('echarts')

The ``ECharts`` pane renders [Apache ECharts](https://echarts.apache.org/en/index.html) and [pyecharts](https://pyecharts.org/#/) plots inside Panel. Note that to use the ``ECharts`` pane in the notebook the Panel extension has to be loaded with 'echarts' as an argument to ensure that echarts.js is initialized. 

#### Parameters:

For layout and styling related parameters see the [customization user guide](../../user_guide/Customization.ipynb).

* **``object``** (dict): An ECharts plot specification expressed as a Python dictionary, which is then converted to JSON. Or a pyecharts chart like `pyecharts.charts.Bar`.
* **``renderer``** (str): Whether to render with HTML 'canvas' (default) or 'svg'
* **``theme``** (str): Theme to apply to plots (one of 'default', 'dark', 'light')
___

Lets try the ``ECharts`` pane support for ECharts specs in its raw form (i.e. a dictionary), e.g. here we declare a bar plot:

In [ ]:
echart = {
    'title': {
        'text': 'ECharts entry example'
    },
    'tooltip': {},
    'legend': {
        'data':['Sales']
    },
    'xAxis': {
        'data': ["shirt","cardign","chiffon shirt","pants","heels","socks"]
    },
    'yAxis': {},
    'series': [{
        'name': 'Sales',
        'type': 'bar',
        'data': [5, 20, 36, 10, 10, 20]
    }],
};
echart_pane = pn.pane.ECharts(echart, height=480, width=640)
echart_pane

Like all other panes, the ``ECharts`` pane ``object`` can be updated, either in place and triggering an update:

In [ ]:
echart['series'] = [dict(echart['series'][0], type= 'line')]
echart_pane.param.trigger('object')

Vega specification can also be responsively sized by declaring the width or height to match the container:

In [ ]:
responsive_spec = dict(echart, responsive=True)

pn.pane.ECharts(responsive_spec, height=400)

The ECharts pane also hass support for pyecharts. For example, we can pass a `pyecharts.charts.Bar` chart directly the `ECharts` pane.  

In [ ]:
from pyecharts.charts import Bar

bar1 = pn.widgets.IntSlider(start=1, end=100, value=50)
bar2 = pn.widgets.IntSlider(start=1, end=100, value=50)

@pn.depends(bar1.param.value, bar2.param.value)
def plot(bar1, bar2):
    my_plot= (Bar()
        .add_xaxis(['Helicoptors', 'Planes'])
        .add_yaxis('Total In Flight', [bar1, bar2])
    )
    return pn.pane.ECharts(my_plot, width=500, height=250)
pn.Row(pn.Column(bar1, bar2), plot).servable()

The ECharts library supports a wide range of chart types and since the plots are expressed using JSON datastructures we can easily update the data and then emit change events to update the charts:

In [ ]:
gauge = {
    'tooltip': {
        'formatter': '{a} <br/>{b} : {c}%'
    },
    'series': [
        {
            'name': 'Gauge',
            'type': 'gauge',
            'detail': {'formatter': '{value}%'},
            'data': [{'value': 50, 'name': 'Value'}]
        }
    ]
};
gauge_pane = pn.pane.ECharts(gauge, width=400, height=400)

slider = pn.widgets.IntSlider(start=0, end=100)

slider.jscallback(args={'gauge': gauge_pane}, value="""
gauge.data.series[0].data[0].value = cb_obj.value
gauge.properties.data.change.emit()
""")

pn.Column(slider, gauge_pane)

### Controls

The `EChart` pane exposes a number of options which can be changed from both Python and Javascript. Try out the effect of these parameters interactively:

In [ ]:
pn.Row(gauge_pane.controls(jslink=True), gauge_pane)